In [1]:
#pip install --no-cache-dir jupyter langchain_openai langchain_community langchain langgraph faiss-cpu sentence-transformers ipywidgets transformers nltk scikit-learn matplotlib markdown langchain_chroma

import yaml
import ipywidgets as widgets
from IPython.display import display
import pickle
import markdown
import glob
import re
import os

In [3]:
import logging
from story_sage.story_sage import StorySage

# Configure the logger

logger = logging.getLogger('story_sage')
logger.setLevel(logging.DEBUG)
# Create a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it for the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(console_handler)

# Filter out logs from other modules
class StorySageFilter(logging.Filter):
    def filter(self, record):
        return record.name.startswith('story_sage')

logger.addFilter(StorySageFilter())



with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

api_key = config['OPENAI_API_KEY']
chroma_path = config['CHROMA_PATH']
chroma_collection = config['CHROMA_COLLECTION']

# Load series.yml to create a mapping from series_metadata_name to series_id
with open('series.yml', 'r') as file:
    series_list = yaml.safe_load(file)
metadata_to_id = {series['series_metadata_name']: series['series_id'] for series in series_list}

# Load all character dictionaries and merge them using the metadata_to_id mapping
# Load entities.json
with open('entities.json', 'r') as file:
    entities = yaml.safe_load(file)

story_sage = StorySage(
    api_key=api_key,
    chroma_path=chroma_path,
    chroma_collection_name=chroma_collection,
    entities=entities,
    series_yml_path='series.yml',
    n_chunks=10
)


# Add a handler to the StorySage logger
story_sage.logger = logger

def invoke_story_sage(data: dict):
    required_keys = ['question', 'book_number', 'chapter_number', 'series_id']
    if not all(key in data for key in required_keys):
        return {'error': f'Missing parameter! Request must include {", ".join(required_keys)}'}, 400

    try:
        result, context = story_sage.invoke(**data)
        return result, context
    except Exception as e:
        raise e
        return {'error': 'Internal server error.'}, 500


2024-12-18 07:14:34,142 - story_sage.story_sage - DEBUG - Logger initialized from parent.
2024-12-18 07:14:34,142 - story_sage.story_sage - DEBUG - Logger initialized from parent.


In [3]:
data = {
    'question': "what is the name of rand's horse?",
    'book_number': 2,
    'chapter_number': 1,
    'series_id': 3
}

if False:
    response, context = invoke_story_sage(data)
    print(response)

2024-12-16 22:14:32,618 - story_sage.story_sage_retriever - INFO - {'$and': [{'$or': [{'book_number': {'$lt': 2}}, {'$and': [{'book_number': 2}, {'chapter_number': {'$lt': 1}}]}]}, {'$and': [{'p_3_6': True}, {'a_3_19': True}]}]}


- Rand's horse is named Bela. 
  - "Rand moved his horse close to Bela and touched her shoulder." (Book 1, Chapter 45)


In [4]:
import chromadb
from story_sage.story_sage_embedder import StorySageEmbedder
from langchain.embeddings import SentenceTransformerEmbeddings
class EmbeddingAdapter(SentenceTransformerEmbeddings):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _embed_documents(self, texts):
        return super().embed_documents(texts)  

    def __call__(self, input):
        return self._embed_documents(input)  

embedder = EmbeddingAdapter
client = chromadb.PersistentClient(path=chroma_path)
vector_store = client.get_collection(name=chroma_collection)

In [5]:
filter_dict = {'$and': [
                {'$or': [
                    {'book_number': {'$lt': 1}},
                    {'$and': [
                        {'book_number': 1}, 
                        {'chapter_number': {'$lt': 25}}
                    ]}
                ]}, 
                {'a_3_12': True}
               ]}

filter_dict = {'$or': [
                    {'book_number': {'$lt': 1}},
                    {'$and': [
                        {'book_number': 1}, 
                        {'chapter_number': {'$lt': 25}}
                    ]}
                ]}
#client.delete_collection('wot_retriever_test')
vector_store.query(query_texts=['harry'],
                   n_results=5,
                   where=filter_dict,
                   include=['metadatas','documents'])

{'ids': [['3_1_4_25', '3_1_24_12', '3_1_2_15', '3_1_7_15', '3_1_21_29']],
 'embeddings': None,
 'documents': [['“I want to see you eat fire.” “The harp!” a voice cried from the crowd. “Play the harp!” Someone else called for the flute. At that moment the door of the inn opened and the Village Council trundled out, Nynaeve in their midst. Padan Fain was not with them, Rand saw; apparently the peddler had decided to remain in the warm common room with his mulled wine. Muttering about “a strong brandy,” Thom Merrilin abruptly jumped down from the old foundation. He ignored the cries of those who had been watching him, pressing inside past the Councilors before they were well out of the doorway. “Is he supposed to be a gleeman or a king?” Cenn Buie asked in annoyed tones. “A waste of good money, if you ask me.” Bran al’Vere half turned after the gleeman, then shook his head. “That man may be more trouble than he’s worth.” Nynaeve, busy gathering her cloak around her, sniffed loudly. “Worry

In [5]:
data = {
    'question': 'Explain the interactions between Cenn and Rand',
    'book_number': 2,
    'chapter_number': 1,
    'series_id': 3
}

response, context = invoke_story_sage(data)
print(response)

2024-12-16 20:31:30,984 - story_sage - DEBUG - Invoking StorySage with question: Explain the interactions between Cenn and Rand, book_number: 2, chapter_number: 1, series_id: 3
2024-12-16 20:31:30,984 - story_sage - DEBUG - Invoking StorySage with question: Explain the interactions between Cenn and Rand, book_number: 2, chapter_number: 1, series_id: 3
2024-12-16 20:31:30,986 - story_sage.story_sage_chain - DEBUG - Extracting characters from question.
2024-12-16 20:31:30,986 - story_sage.story_sage_chain - DEBUG - Extracting characters from question.
2024-12-16 20:31:30,987 - story_sage.story_sage_chain - DEBUG - Series ID found in state.
2024-12-16 20:31:30,987 - story_sage.story_sage_chain - DEBUG - Series ID found in state.


{'error': 'Internal server error.'}


In [5]:
# Define the input and output widgets
input_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    continuous_update=False,
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to submit your question',
    icon='check'
)

book_number_box = widgets.IntText(
    value=10,
    description='Book Number:',
    disabled=False
)

chapter_number_box = widgets.IntText(
    value=None,
    description='Chapter Number:',
    disabled=False
)

status_box = widgets.Output(layout={'min_height': '50px'})
output_box = widgets.Output(layout={'min_height': '200px'})
context_box = widgets.Output(layout={'min_height': '200px'})

# Create a spinner widget
spinner = widgets.HTML(
    value="""<i class="fa fa-spinner fa-spin" style="font-size:24px; color:#2a9df4;"></i>""",
    placeholder='Loading...',
    description=''
)

# Make sure Font Awesome is available
display(widgets.HTML("<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css'>"))

def wrap_answer(answer):
    html = markdown.markdown(answer)
    return f"<div style='background-color: #f9f9f9; padding: 10px; border-radius: 5px;'>{html}</div>"

def show_results(answer, context):
    with output_box:
        output_box_contents = []
        output_box_contents.append("<h3>Answer</h3>")
        output_box_contents.append(wrap_answer(answer))
        display(widgets.HTML(''.join(output_box_contents)))
        with context_box:
            context_box.clear_output()
            context_box_contents = []
            context_box_contents.append("<h3>Context</h3>")
            for idx in range(len(context['metadatas'])):
                meta = context['metadatas'][0][idx]
                content = context['documents'][0][idx]
                context_box_contents.append(f"<p><strong>Book Number:</strong> {meta['book_number']} <strong>Chapter Number:</strong> {meta['chapter_number']}</p>")
                context_box_contents.append(f"<p>{content}</p>")
            display(widgets.HTML(wrap_answer("".join(context_box_contents))))

    

# Define the function to handle the button click
def submit_question(b):
    with status_box:
        status_box.clear_output()
        display(widgets.HTML(f"<h3>Retrieving top relevant chunks...</h3>"))
        with output_box:
            output_box.clear_output()
            display(spinner)
            answer, context = story_sage.invoke(
                question = input_box.value,
                book_number = book_number_box.value,
                chapter_number = chapter_number_box.value
            )
            output_box.clear_output()
    show_results(answer, context)

# Attach the handler to the button
submit_button._click_handlers.callbacks.clear()
submit_button.on_click(submit_question)

# Attach the handler to the input box for the return key
#input_box.observe(submit_question)

# Display the widgets
display(status_box, book_number_box, chapter_number_box, input_box, submit_button, output_box, context_box)

HTML(value="<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-aw…

Output(layout=Layout(min_height='50px'))

IntText(value=10, description='Book Number:')

IntText(value=0, description='Chapter Number:')

Text(value='', continuous_update=False, description='Question:', placeholder='Type your question here...')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit your question')

Output(layout=Layout(min_height='200px'))

Output(layout=Layout(min_height='200px'))